In [ ]:
#https://www.kaggle.com/code/yonatanrabinovich/anime-recommendations-project/notebook
#cosine similarity between user_id, name, rating to recomend similar animes that you already watched
#https://towardsdatascience.com/using-cosine-similarity-to-build-a-movie-recommendation-system-ae7f20842599

In [1]:
import pandas as pd
import numpy as np
import matplotlib as plt
import seaborn as sns
import scipy as sp 
from sklearn.feature_extraction.text import CountVectorizer
from sklearn import preprocessing 
#ML model
from sklearn.metrics.pairwise import cosine_similarity

In [2]:
df_anime = pd.read_csv('/Users/Simon/OneDrive - Sistemas Expertos S.A.S/PC SIMON/Programacion/python/datasets/anime_database/anime.csv')
df_rating = pd.read_csv('/Users/Simon/OneDrive - Sistemas Expertos S.A.S/PC SIMON/Programacion/python/datasets/anime_database/rating.csv')

In [3]:
df_anime['rating_1'] = df_anime['rating']
df_anime

,anime_id,name,genre,type,episodes,rating,members,rating_1
0,32281,Kimi no Na wa.,"Drama, Romance, School, Supernatural",Movie,1,9.37,200630,9.37
1,5114,Fullmetal Alchemist: Brotherhood,"Action, Adventure, Drama, Fantasy, Magic, Mili...",TV,64,9.26,793665,9.26
2,28977,Gintama°,"Action, Comedy, Historical, Parody, Samurai, S...",TV,51,9.25,114262,9.25
3,9253,Steins;Gate,"Sci-Fi, Thriller",TV,24,9.17,673572,9.17
4,9969,Gintama&#039;,"Action, Comedy, Historical, Parody, Samurai, S...",TV,51,9.16,151266,9.16
...,...,...,...,...,...,...,...,...
12289,9316,Toushindai My Lover: Minami tai Mecha-Minami,Hentai,OVA,1,4.15,211,4.15
12290,5543,Under World,Hentai,OVA,1,4.28,183,4.28
12291,5621,Violence Gekiga David no Hoshi,Hentai,OVA,4,4.88,219,4.88
12292,6133,Violence Gekiga Shin David no Hoshi: Inma Dens...,Hentai,OVA,1,4.98,175,4.98


In [4]:
df_anime.isna().sum()

anime_id      0
name          0
genre        62
type         25
episodes      0
rating      230
members       0
rating_1    230
dtype: int64

In [5]:
df_anime = df_anime[~np.isnan(df_anime['rating'])]
df_anime = df_anime[df_anime['type']=='TV']

In [6]:
print(df_anime['genre'].mode())
df_anime['genre'] = df_anime['genre'].fillna('Comedy')

0    Comedy
Name: genre, dtype: object


In [7]:
#rating -1 shows that the anime wasn't voted
df_rating['rating'].value_counts()

rating
 8     1646019
-1     1476496
 7     1375287
 9     1254096
 10     955715
 6      637775
 5      282806
 4      104291
 3       41453
 2       23150
 1       16649
Name: count, dtype: int64

In [8]:
df_rating['rating'] = df_rating['rating'].apply(lambda x: np.nan if x==-1 else x)
df_rating

,user_id,anime_id,rating
0,1,20,NaN
1,1,24,NaN
2,1,79,NaN
3,1,226,NaN
4,1,241,NaN
...,...,...,...
7813732,73515,16512,7.0
7813733,73515,17187,9.0
7813734,73515,22145,10.0
7813735,73516,790,9.0


In [9]:
#to much data for my slow computer, soon i will have a more powerful one :)
#join that 
rated_anime = df_rating.merge(df_anime, left_on = 'anime_id', right_on='anime_id')
#only leaving useful info 
rated_anime = rated_anime[['user_id', 'name', 'rating_1']] 
#reduce num of data so my laptop doesn't go brruuuuummmmmmm
rated_anime_10k = rated_anime[rated_anime.user_id <=10000]
rated_anime_10k

,user_id,name,rating_1
0,1,Naruto,7.81
1,3,Naruto,7.81
2,5,Naruto,7.81
3,6,Naruto,7.81
4,10,Naruto,7.81
...,...,...,...
5281663,9274,Mazinger ZIP!,6.14
5281668,9401,Croket!,7.12
5281672,9591,Magical★Taruruuto-kun,6.80
5281678,9591,Konchuu Monogatari Minashigo Hutch (1989),6.60


In [10]:
pivot = rated_anime_10k.pivot_table(index=['user_id'], columns=['name'], values='rating_1')
pivot.head()

name,.hack//Roots,.hack//Sign,.hack//Tasogare no Udewa Densetsu,009-1,07-Ghost,11eyes,12-sai.: Chicchana Mune no Tokimeki,3 Choume no Tama: Uchi no Tama Shirimasenka?,30-sai no Hoken Taiiku,91 Days,...,"Zone of the Enders: Dolores, I",Zukkoke Knight: Don De La Mancha,ef: A Tale of Melodies.,ef: A Tale of Memories.,gdgd Fairies,gdgd Fairies 2,iDOLM@STER Xenoglossia,s.CRY.ed,xxxHOLiC,xxxHOLiC Kei
user_id,,,,,,,,,,,,,,,,,,,,,
1,NaN,NaN,NaN,NaN,NaN,6.49,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,8.11,NaN


In [11]:
#Value normalization
pivot_n = pivot.apply(lambda x: (x-np.mean(x))/(np.max(x)-np.min(x)), axis=1)
#replacing Nan with 0
pivot_n.fillna(0, inplace=True)
#Transpose
pivot_n = pivot_n.T
#Dropping columns with the values of 0 (unrated)
pivot_n = pivot_n.loc[:, (pivot_n != 0).any(axis=0)]
#Using scipy package to convert to sparse matrix format for the similarity computation
piv_sparse = sp.sparse.csr_matrix(pivot_n.values)

In [12]:
anime_cos_sim = cosine_similarity(piv_sparse)
anime_cos_sim

array([[ 1.        ,  0.48304291,  0.43675576, ...,  0.13292516,
        -0.13574807, -0.11574171],
       [ 0.48304291,  1.        ,  0.44147424, ...,  0.15589698,
        -0.13790196, -0.11840431],
       [ 0.43675576,  0.44147424,  1.        , ...,  0.10497299,
        -0.13367339, -0.10150771],
       ...,
       [ 0.13292516,  0.15589698,  0.10497299, ...,  1.        ,
        -0.06819154, -0.05512292],
       [-0.13574807, -0.13790196, -0.13367339, ..., -0.06819154,
         1.        ,  0.71056194],
       [-0.11574171, -0.11840431, -0.10150771, ..., -0.05512292,
         0.71056194,  1.        ]])

In [13]:
#now making it a df
ani_sim_df = pd.DataFrame(anime_cos_sim, index=pivot_n.index, columns=pivot_n.index)
ani_sim_df

name,.hack//Roots,.hack//Sign,.hack//Tasogare no Udewa Densetsu,009-1,07-Ghost,11eyes,12-sai.: Chicchana Mune no Tokimeki,3 Choume no Tama: Uchi no Tama Shirimasenka?,30-sai no Hoken Taiiku,91 Days,...,"Zone of the Enders: Dolores, I",Zukkoke Knight: Don De La Mancha,ef: A Tale of Melodies.,ef: A Tale of Memories.,gdgd Fairies,gdgd Fairies 2,iDOLM@STER Xenoglossia,s.CRY.ed,xxxHOLiC,xxxHOLiC Kei
name,,,,,,,,,,,,,,,,,,,,,
.hack//Roots,1.000000,0.483043,0.436756,0.035721,0.070056,0.122621,0.004887,0.000000,0.051610,-0.025314,...,0.065054,0.0,-0.113007,-0.108576,0.000000,0.000000,0.065426,0.132925,-0.135748,-0.115742
.hack//Sign,0.483043,1.000000,0.441474,0.035239,0.089239,0.093153,0.003677,0.000000,0.059214,-0.023532,...,0.076632,0.0,-0.111885,-0.122537,0.011767,0.010797,0.048033,0.155897,-0.137902,-0.118404
.hack//Tasogare no Udewa Densetsu,0.436756,0.441474,1.000000,0.064496,0.057389,0.092265,0.005965,0.000000,0.043266,-0.021194,...,0.030686,0.0,-0.119785,-0.129481,0.000000,0.000000,0.070611,0.104973,-0.133673,-0.101508
009-1,0.035721,0.035239,0.064496,1.000000,0.012034,0.061638,0.011157,0.000000,0.002477,-0.023557,...,0.008970,0.0,-0.030307,-0.031157,0.000000,0.000000,0.013543,0.012651,-0.082850,-0.058980
07-Ghost,0.070056,0.089239,0.057389,0.012034,1.000000,0.164189,0.009277,0.000548,0.044063,-0.025039,...,0.009376,0.0,-0.084299,-0.080929,-0.000472,-0.000355,0.021332,0.051022,-0.097623,-0.111422
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
gdgd Fairies 2,0.000000,0.010797,0.000000,0.000000,-0.000355,0.001218,0.000000,0.000000,0.000000,-0.016189,...,0.000000,0.0,-0.003648,-0.007009,0.776453,1.000000,0.006611,0.009750,-0.010336,0.000000
iDOLM@STER Xenoglossia,0.065426,0.048033,0.070611,0.013543,0.021332,0.068340,0.042430,0.000000,0.010935,-0.042748,...,0.018833,0.0,-0.124561,-0.126264,0.008778,0.006611,1.000000,0.003082,-0.089927,-0.063873
s.CRY.ed,0.132925,0.155897,0.104973,0.012651,0.051022,0.023177,0.000000,0.000000,0.011593,-0.009030,...,0.026179,0.0,-0.015562,-0.027255,0.010670,0.009750,0.003082,1.000000,-0.068192,-0.055123


In [14]:
def anime_recommendation(ani_name):
    num = 1
    print(f'Recomendations because you watched: {ani_name}')
    for anime in ani_sim_df.sort_values(by=ani_name, ascending=False).index[1:6]:
        print(f'{num}: {anime}, {round(ani_sim_df[anime][ani_name]*100,2)}% match')
        num +=1

In [15]:
anime_recommendation('Naruto')

Recomendations because you watched: Naruto
1: Sword Art Online, 28.65% match
2: Elfen Lied, 25.81% match
3: Bleach, 25.39% match
4: Ao no Exorcist, 24.72% match
5: Shaman King, 21.2% match
